In [ ]:
!pip install -r ../requirements.txt

In [1]:
import sys
sys.path.insert(0, "../")

# Downloading and processing data

There are two ways you can obtain OSM data in osm.pbf format:  
- Download it yourself: [parts of the world](https://download.geofabrik.de/), [cities](https://download.bbbike.org/osm/bbbike/), [adjustable area](https://extract.bbbike.org/) (via mail), [adjustable area](https://export.hotosm.org/en/v3/) (online), [planet](https://planet.maps.mail.ru/pbf/)
- Let the program download it for you

If the map is downloaded you can specify the filename:

In [ ]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
filename = "../maps/kozlovo.osm.pbf"
bbox = [36.2, 56.5, 36.7, 56.7]
vgraph.compute_geometry(bbox=bbox, filename=filename)

Or, alternatively, you can only specify the bounding box, and the map will be downloaded automatically ([curl](https://curl.se/) & [osmosis](https://wiki.openstreetmap.org/wiki/Osmosis) required):

In [ ]:
bbox = [34, 59, 34.2, 59.1]
vgraph.compute_geometry(bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen or default parameters.  
If not specified, optimal parameters will be computed by the algorithm.

In [ ]:
vgraph.prune_geometry(epsilon_polygon=0.003,
                       epsilon_linestring=0.001,
                       bbox_comp=10)

Computed data can also be saved in .h5 file to skip data processing the next time:

In [ ]:
vgraph.save_geometry("../maps/user_area.h5")

# Using precomputed data and building visibility graph

In [2]:
from visibility.visibility_graph import VisibilityGraph

vgraph = VisibilityGraph()
vgraph.load_geometry("../maps/user_area.h5")

Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet:

In [3]:
%%time
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = vgraph.build_graph(inside_percent=0.4,
                            multiprocessing=False,
                            graph=True,
                            map_plot=map_plot)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.60it/s]


edges:  1420
nodes:  191
Wall time: 6.62 s


c:\users\kozub\offroad-routing-engine\venv\lib\site-packages\IPython\core\display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


VisibilityGraph may also be used to find incident edges for a single point.  
This feature is used for pathfinding without graph building:

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ((34.02, 59.01), None, None, None, None)
incidents = vgraph.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)